pip install ollama

ollama run llama3.2
ollama pull llama3.2

In [1]:
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv())


True

In [2]:
from pathlib import Path
path_output = os.getcwd()

In [3]:
#from llamacpp_model import LlamaModel
from IPython.display import Markdown
import pandas as pd
import json
import re
from ollama import chat,ChatResponse

## Open the isear dataset

In [4]:
df_isear = pd.read_csv('isear_dataset.csv')
df_isear.head()

,id,text,emotion,intensity,gender,age,predicted_emotion_llama32,confidence_score_llama32,predicted_emotion_openai,confidence_score_openai,confidence_score_anthropic,predicted_emotion_anthropic,confidence_score_llama3_3,predicted_emotion_llama3_3,predicted_emotion_qwen,confidence_score_qwen,predicted_emotion_cohere,confidence_score_cohere,predicted_emotion_deepseek,confidence_score_deepseek
0,0,"During the period of falling in love, each tim...",joy,3,male,33,joy,0.85,joy,0.90,0.85,joy,0.8,joy,joy,0.95,joy,0.9,joy,0.95
1,1,When I was involved in a traffic accident.,fear,2,male,33,fear,0.80,fear,0.80,0.85,fear,0.8,fear,fear,0.90,fear,0.9,guilt,0.95
2,2,When I was driving home after several days of...,anger,3,male,33,anger,0.81,anger,0.85,0.85,anger,0.8,anger,anger,0.80,anger,0.9,anger,0.95
3,3,When I lost the person who meant the most to me.,sadness,4,male,33,sadness,0.87,sadness,0.95,0.95,sadness,0.9,sadness,sadness,0.95,sadness,1.0,sadness,0.95
4,4,The time I knocked a deer down - the sight of ...,disgust,4,male,33,sadness,0.90,sadness,0.90,0.90,sadness,0.8,disgust,sadness,0.95,sadness,0.9,guilt,0.95


In [ ]:
df_fp = df_isear[df_isear['emotion'] != df_isear['predicted_emotion_llama32']]
for i, i_df in df_fp.groupby(by=['emotion']):
    print(i, i_df.shape)

In [13]:
for i, i_df in df_fp.groupby(by=['emotion']):
    print(i, i_df.shape)

('anger',) (454, 20)
('disgust',) (423, 20)
('fear',) (208, 20)
('guilt',) (308, 20)
('joy',) (135, 20)
('sadness',) (78, 20)
('shame',) (186, 20)


In [8]:
df_fp.describe()

,id,intensity,age,confidence_score_llama32,confidence_score_openai,confidence_score_anthropic,confidence_score_llama3_3,confidence_score_qwen,confidence_score_cohere,confidence_score_deepseek
count,1792.000000,1792.000000,1792.000000,1792.000000,1792.000000,1792.000000,1792.000000,1792.000000,1792.000000,1792.000000
mean,3691.247210,2.789621,22.487723,0.839850,0.833036,0.775536,0.749609,0.850056,0.808064,0.093359
std,2128.520138,0.992346,3.910339,0.073267,0.094825,0.227106,0.184339,0.107338,0.082571,0.265266
min,4.000000,0.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1802.750000,2.000000,20.000000,0.800000,0.800000,0.850000,0.800000,0.800000,0.800000,0.000000
50%,3797.000000,3.000000,21.000000,0.800000,0.850000,0.850000,0.800000,0.850000,0.800000,0.000000
75%,5460.500000,4.000000,24.000000,0.900000,0.900000,0.850000,0.800000,0.900000,0.900000,0.000000
max,7283.000000,4.000000,35.000000,1.000000,1.000000,0.950000,1.000000,0.950000,0.900000,1.000000


### Llama 3.2

In [16]:
for index, row in df_isear.iterrows():
    # Construct the prompt dynamically
    prompt = f"""
        # Emotion Analysis Prompt

        You are an expert in emotion analysis. Given a text input, classify its **primary emotion** strictly as one of the following: **'joy', 'fear', 'anger', 'sadness', 'disgust', 'shame',** or **'guilt'**. **Do not use any other emotions.**
        ---
        ## Instructions

        1. **Read and understand** the input text carefully.
        2. **Strictly classify** the text as one of these **only**: 'joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', or 'guilt'.
        ---

        ## Definitions and Examples

        ### Joy
        - **Definition:** A feeling of happiness, excitement, or pleasure.
        - **Examples:**
        - Receiving good news, celebrating success, spending time with loved ones.
        - Enjoying a meaningful moment with friends or family.
        - **Refinement:** If the text expresses relief rather than happiness, classify it as **joy**.

        ### Fear
        - **Definition:** A response to perceived danger, risk, or uncertainty, often accompanied by anxiety, nervousness, or distress.
        - **Examples:**
        - Experiencing a sudden scare or physical danger (e.g., a traffic accident, being chased).
        - Worrying about an uncertain future or an important exam.
        - Feeling vulnerable in an unfamiliar or threatening environment.
        - Hearing about a serious illness or personal risk (e.g., medical tests, a pandemic).
        - Imagining a loved one being in danger.
        - Being caught in a situation where you have **no control**.
        - **Refinements:**
        - If the emotion is about loss, classify as **sadness**.
        - If the emotion is about remorse for past actions, classify as **guilt**.

        ### Anger
        - **Definition:** A feeling of irritation, frustration, or rage in response to a perceived injustice, wrongdoing, or moral outrage.
        - **Examples:**
        - Witnessing someone being treated unfairly.
        - Feeling ignored, rejected, or ridiculed.
        - Seeing cruelty, unethical behavior, or injustice.
        - Facing obstacles due to someone else's irresponsibility.
        - Experiencing personal attacks or insults.
        - Dealing with broken promises or betrayal.
        - Encountering discrimination or invasion of personal boundaries.
        - **Refinement:** If the text shows resentment or frustration, classify as **anger**; if it expresses grief or emotional pain, classify as **sadness**.

        ### Sadness
        - **Definition:** A feeling of deep sorrow, grief, or loss, often associated with disappointment, longing, or emotional pain.
        - **Examples:**
        - Losing a loved one, pet, or cherished relationship.
        - Feeling lonely or disconnected.
        - Experiencing personal failure or missed opportunities.
        - Witnessing suffering or misfortune of others.
        - Regretting something that cannot be changed.
        - Feeling rejected or abandoned.
        - Nostalgia for better times.
        - Social isolation or exclusion.
        - Empathy for others' suffering.
        - **Refinements:**
        - If the emotion is regret about one’s own actions, classify as **guilt**.
        - If it is about loss or despair, classify as **sadness**.
        - If the response is due to injustice or frustration, classify as **anger**.

        ### Disgust
        - **Definition:** A strong aversion, repulsion, or deep-seated rejection toward something perceived as offensive, impure, unethical, or physically revolting.
        - **Examples:**
        - Seeing animals or people being mistreated.
        - Reading about unethical political actions or human rights violations.
        - Witnessing something grotesque or disturbing.
        - Feeling sickened by someone's behavior (e.g., lying, cheating, betrayal).
        - Encountering unclean, unhygienic, or repulsive physical conditions.
        - **Refinements:**
        - If the reaction is about personal danger, classify as **fear**.
        - If it is about moral or physical revulsion, classify as **disgust**.
        - If the feeling is self-directed regarding a moral failing, classify as **guilt**.

        ### Shame
        - **Definition:** A feeling of humiliation or distress caused by the awareness of wrongdoing or foolish behavior, often accompanied by embarrassment, dishonor, or exposure to judgment.
        - **Examples:**
        - Feeling exposed or judged for personal flaws or mistakes.
        - Experiencing social disapproval or loss of reputation.
        - Being caught in acts of dishonesty or deceit.
        - Violating personal or cultural values, leading to feelings of unworthiness.
        - **Refinements:**
        - If the emotion focuses on feeling judged or humiliated without explicit remorse for harm, classify as **shame**.
        - If the emotion includes explicit remorse for harming someone, classify as **guilt**.

        ### Guilt
        - **Definition:** A feeling of remorse or regret over actions that have harmed others or violated moral principles, focused on personal responsibility rather than external judgment.
        - **Examples:**
        - Feeling bad for hurting someone’s feelings.
        - Regretting a past mistake or unethical action.
        - Realizing that one’s negligence caused harm.
        - Breaking a rule and later feeling remorseful.
        - **Refinements:**
        - Only classify as **guilt** if there is explicit harm or clear wrongdoing causing remorse.
        - If there is no explicit harm mentioned, default to **shame**.
        - Increase the threshold for guilt; use it only when clear personal responsibility for harm is evident.

        ---

        ## Additional Guidelines:
        - **Default to Shame:** When the text does not explicitly show remorse for an action but hints at personal inadequacy, classify as **shame**.
        - **Mixed Cases:** Evaluate the overall tone and select the most prominent emotion.
        - **Confidence Scoring:** Assign a confidence score between `0` and `1` based on your certainty. If confidence is below `0.6`, lean toward **shame** or **sadness**.
        - **Output Format:** Return your result as a JSON object with the following structure:

        ```json
        {{
        "emotion": "one_of_the_specified_emotions",
        "confidence_score": confidence_value
        }}


        Now, analyze the text - {row.text} and output only the JSON response with no additional formatting, code blocks, or explanations.
        """
    labels = ['joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', 'guilt']
    
    if True: #row['predicted_emotion_llama32'].lower()=='unknown':# not in labels or row['emotion']!= row['predicted_emotion_llama32'].lower():
        messages = [
            {
                'role': 'user',
                'content': prompt,
            }
        ]
        try:
            response: ChatResponse = chat(model='llama3.2', messages=messages, options={"temperature":0.3} )
            content = response['message']['content']
            # Check if content is wrapped in markdown-like code block
            if content.startswith("```json") and content.endswith("```"):
                content = re.sub(r'^```json\s*|\s*```$', '', content).strip()
            # Parse the JSON response correctly
            predicted_emotion = json.loads(content)
            predicted_emotion_label  = "unknown"
            predicted_emotion_score = 0.0
            if predicted_emotion.get("emotion").lower() in labels:
                # Store the results in the DataFrame
                predicted_emotion_label = predicted_emotion.get('emotion', 'unknown').lower()
                predicted_emotion_score = predicted_emotion.get('confidence_score', 0.0)   
            df_isear.at[index, 'predicted_emotion_llama32'] = predicted_emotion_label
            df_isear.at[index, 'confidence_score_llama32'] = predicted_emotion_score 
        except:
            predicted_emotion_label = 'unknown'
            df_isear.at[index, 'predicted_emotion_llama32'] = predicted_emotion_label
            df_isear.at[index, 'confidence_score_llama32'] = 0
        
        print(f"Row {index}| Ground truth - {row.emotion} and predicted -{predicted_emotion_label}")
        if index % 500==0:
            df_isear.to_csv(Path(path_output) / 'isear_dataset_new.csv', index=False)
df_isear.to_csv(Path(path_output) / 'isear_dataset_new.csv', index=False)

Row 0| Ground truth - joy and predicted -sadness
Row 1| Ground truth - fear and predicted -fear
Row 2| Ground truth - anger and predicted -anger
Row 3| Ground truth - sadness and predicted -sadness
Row 4| Ground truth - disgust and predicted -sadness
Row 5| Ground truth - shame and predicted -guilt
Row 6| Ground truth - guilt and predicted -guilt
Row 7| Ground truth - joy and predicted -joy
Row 8| Ground truth - fear and predicted -fear
Row 9| Ground truth - anger and predicted -sadness
Row 10| Ground truth - sadness and predicted -shame
Row 11| Ground truth - disgust and predicted -disgust
Row 12| Ground truth - shame and predicted -shame
Row 13| Ground truth - guilt and predicted -fear
Row 14| Ground truth - joy and predicted -joy
Row 15| Ground truth - fear and predicted -fear
Row 16| Ground truth - anger and predicted -shame
Row 17| Ground truth - sadness and predicted -guilt
Row 18| Ground truth - disgust and predicted -shame
Row 19| Ground truth - shame and predicted -guilt
Row 2

## Qwen Model

In [ ]:
for index, row in df_isear.iterrows():
    # Construct the prompt dynamically
    prompt = f"""
        # Emotion Analysis Prompt

        You are an expert in emotion analysis. Given a text input, classify its **primary emotion** strictly as one of the following: **'joy', 'fear', 'anger', 'sadness', 'disgust', 'shame',** or **'guilt'**. **Do not use any other emotions.**
        ---
        ## Instructions

        1. **Read and understand** the input text carefully.
        2. **Strictly classify** the text as one of these **only**: 'joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', or 'guilt'.
        ---

        ## Definitions and Examples

        ### Joy
        - **Definition:** A feeling of happiness, excitement, or pleasure.
        - **Examples:**
        - Receiving good news, celebrating success, spending time with loved ones.
        - Enjoying a meaningful moment with friends or family.
        - **Refinement:** If the text expresses relief rather than happiness, classify it as **joy**.

        ### Fear
        - **Definition:** A response to perceived danger, risk, or uncertainty, often accompanied by anxiety, nervousness, or distress.
        - **Examples:**
        - Experiencing a sudden scare or physical danger (e.g., a traffic accident, being chased).
        - Worrying about an uncertain future or an important exam.
        - Feeling vulnerable in an unfamiliar or threatening environment.
        - Hearing about a serious illness or personal risk (e.g., medical tests, a pandemic).
        - Imagining a loved one being in danger.
        - Being caught in a situation where you have **no control**.
        - **Refinements:**
        - If the emotion is about loss, classify as **sadness**.
        - If the emotion is about remorse for past actions, classify as **guilt**.

        ### Anger
        - **Definition:** A feeling of irritation, frustration, or rage in response to a perceived injustice, wrongdoing, or moral outrage.
        - **Examples:**
        - Witnessing someone being treated unfairly.
        - Feeling ignored, rejected, or ridiculed.
        - Seeing cruelty, unethical behavior, or injustice.
        - Facing obstacles due to someone else's irresponsibility.
        - Experiencing personal attacks or insults.
        - Dealing with broken promises or betrayal.
        - Encountering discrimination or invasion of personal boundaries.
        - **Refinement:** If the text shows resentment or frustration, classify as **anger**; if it expresses grief or emotional pain, classify as **sadness**.

        ### Sadness
        - **Definition:** A feeling of deep sorrow, grief, or loss, often associated with disappointment, longing, or emotional pain.
        - **Examples:**
        - Losing a loved one, pet, or cherished relationship.
        - Feeling lonely or disconnected.
        - Experiencing personal failure or missed opportunities.
        - Witnessing suffering or misfortune of others.
        - Regretting something that cannot be changed.
        - Feeling rejected or abandoned.
        - Nostalgia for better times.
        - Social isolation or exclusion.
        - Empathy for others' suffering.
        - **Refinements:**
        - If the emotion is regret about one’s own actions, classify as **guilt**.
        - If it is about loss or despair, classify as **sadness**.
        - If the response is due to injustice or frustration, classify as **anger**.

        ### Disgust
        - **Definition:** A strong aversion, repulsion, or deep-seated rejection toward something perceived as offensive, impure, unethical, or physically revolting.
        - **Examples:**
        - Seeing animals or people being mistreated.
        - Reading about unethical political actions or human rights violations.
        - Witnessing something grotesque or disturbing.
        - Feeling sickened by someone's behavior (e.g., lying, cheating, betrayal).
        - Encountering unclean, unhygienic, or repulsive physical conditions.
        - **Refinements:**
        - If the reaction is about personal danger, classify as **fear**.
        - If it is about moral or physical revulsion, classify as **disgust**.
        - If the feeling is self-directed regarding a moral failing, classify as **guilt**.

        ### Shame
        - **Definition:** A feeling of humiliation or distress caused by the awareness of wrongdoing or foolish behavior, often accompanied by embarrassment, dishonor, or exposure to judgment.
        - **Examples:**
        - Feeling exposed or judged for personal flaws or mistakes.
        - Experiencing social disapproval or loss of reputation.
        - Being caught in acts of dishonesty or deceit.
        - Violating personal or cultural values, leading to feelings of unworthiness.
        - **Refinements:**
        - If the emotion focuses on feeling judged or humiliated without explicit remorse for harm, classify as **shame**.
        - If the emotion includes explicit remorse for harming someone, classify as **guilt**.

        ### Guilt
        - **Definition:** A feeling of remorse or regret over actions that have harmed others or violated moral principles, focused on personal responsibility rather than external judgment.
        - **Examples:**
        - Feeling bad for hurting someone’s feelings.
        - Regretting a past mistake or unethical action.
        - Realizing that one’s negligence caused harm.
        - Breaking a rule and later feeling remorseful.
        - **Refinements:**
        - Only classify as **guilt** if there is explicit harm or clear wrongdoing causing remorse.
        - If there is no explicit harm mentioned, default to **shame**.
        - Increase the threshold for guilt; use it only when clear personal responsibility for harm is evident.

        ---

        ## Additional Guidelines:
        - **Default to Shame:** When the text does not explicitly show remorse for an action but hints at personal inadequacy, classify as **shame**.
        - **Mixed Cases:** Evaluate the overall tone and select the most prominent emotion.
        - **Confidence Scoring:** Assign a confidence score between `0` and `1` based on your certainty. If confidence is below `0.6`, lean toward **shame** or **sadness**.
        - **Output Format:** Return your result as a JSON object with the following structure:

        ```json
        {{
        "emotion": "one_of_the_specified_emotions",
        "confidence_score": confidence_value
        }}


        Now, analyze the text - {row.text} and output only the JSON response with no additional formatting, code blocks, or explanations.
        """
    labels = ['joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', 'guilt']
    if row['predicted_emotion_qwen'].lower() =='unknown': #not in labels or row['emotion']!= row['predicted_emotion_qwen'].lower():
        messages = [
            {
                'role': 'user',
                'content': prompt,
            }
        ]
        try:
            response: ChatResponse = chat(model='qwen2.5', messages=messages, options={"temperature":0.3} )
            content = response['message']['content']
            # Check if content is wrapped in markdown-like code block
            if content.startswith("```json") and content.endswith("```"):
                content = re.sub(r'^```json\s*|\s*```$', '', content).strip()
            # Parse the JSON response correctly
            predicted_emotion = json.loads(content)
            predicted_emotion_label  = "unknown"
            predicted_emotion_score = 0.0
            if predicted_emotion.get("emotion").lower() in labels:
                # Store the results in the DataFrame
                predicted_emotion_label = predicted_emotion.get('emotion', 'unknown').lower()
                predicted_emotion_score = predicted_emotion.get('confidence_score', 0.0)   
            df_isear.at[index, 'predicted_emotion_qwen'] = predicted_emotion_label
            df_isear.at[index, 'confidence_score_qwen'] = predicted_emotion_score 
        except:
            df_isear.at[index, 'predicted_emotion_qwen'] = 'unknown'
            df_isear.at[index, 'confidence_score_qwen'] = 0
        
        print(f"Row {index}| Ground truth - {row.emotion} and predicted -{predicted_emotion_label}")
        if index % 500==0:
            df_isear.to_csv(path_output, index=False)
df_isear.to_csv(path_output, index=False)

## Deep Seek model

In [ ]:
for index, row in df_isear[5999:7000].iterrows():
    # Construct the prompt dynamically
    prompt = f"""
        # Emotion Analysis Prompt

        You are an expert in emotion analysis. Given a text input, classify its **primary emotion** strictly as one of the following: **'joy', 'fear', 'anger', 'sadness', 'disgust', 'shame',** or **'guilt'**. **Do not use any other emotions.**
        ---
        ## Instructions

        1. **Read and understand** the input text carefully.
        2. **Strictly classify** the text as one of these **only**: 'joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', or 'guilt'.
        3. **Do not include any explanations, reasoning, or intermediate thoughts.**
        4. **Output must be in JSON format only**, without code blocks, additional formatting, or `<think>` tags.
        5. **Ensure the response consists only of the final JSON result.**
        ---

        ## Definitions and Examples

        ### Joy
        - **Definition:** A feeling of happiness, excitement, or pleasure.
        - **Examples:**
        - Receiving good news, celebrating success, spending time with loved ones.
        - Enjoying a meaningful moment with friends or family.
        - **Refinement:** If the text expresses relief rather than happiness, classify it as **joy**.

        ### Fear
        - **Definition:** A response to perceived danger, risk, or uncertainty, often accompanied by anxiety, nervousness, or distress.
        - **Examples:**
        - Experiencing a sudden scare or physical danger (e.g., a traffic accident, being chased).
        - Worrying about an uncertain future or an important exam.
        - Feeling vulnerable in an unfamiliar or threatening environment.
        - Hearing about a serious illness or personal risk (e.g., medical tests, a pandemic).
        - Imagining a loved one being in danger.
        - Being caught in a situation where you have **no control**.
        - **Refinements:**
        - If the emotion is about loss, classify as **sadness**.
        - If the emotion is about remorse for past actions, classify as **guilt**.

        ### Anger
        - **Definition:** A feeling of irritation, frustration, or rage in response to a perceived injustice, wrongdoing, or moral outrage.
        - **Examples:**
        - Witnessing someone being treated unfairly.
        - Feeling ignored, rejected, or ridiculed.
        - Seeing cruelty, unethical behavior, or injustice.
        - Facing obstacles due to someone else's irresponsibility.
        - Experiencing personal attacks or insults.
        - Dealing with broken promises or betrayal.
        - Encountering discrimination or invasion of personal boundaries.
        - **Refinement:** If the text shows resentment or frustration, classify as **anger**; if it expresses grief or emotional pain, classify as **sadness**.

        ### Sadness
        - **Definition:** A feeling of deep sorrow, grief, or loss, often associated with disappointment, longing, or emotional pain.
        - **Examples:**
        - Losing a loved one, pet, or cherished relationship.
        - Feeling lonely or disconnected.
        - Experiencing personal failure or missed opportunities.
        - Witnessing suffering or misfortune of others.
        - Regretting something that cannot be changed.
        - Feeling rejected or abandoned.
        - Nostalgia for better times.
        - Social isolation or exclusion.
        - Empathy for others' suffering.
        - **Refinements:**
        - If the emotion is regret about one’s own actions, classify as **guilt**.
        - If it is about loss or despair, classify as **sadness**.
        - If the response is due to injustice or frustration, classify as **anger**.

        ### Disgust
        - **Definition:** A strong aversion, repulsion, or deep-seated rejection toward something perceived as offensive, impure, unethical, or physically revolting.
        - **Examples:**
        - Seeing animals or people being mistreated.
        - Reading about unethical political actions or human rights violations.
        - Witnessing something grotesque or disturbing.
        - Feeling sickened by someone's behavior (e.g., lying, cheating, betrayal).
        - Encountering unclean, unhygienic, or repulsive physical conditions.
        - **Refinements:**
        - If the reaction is about personal danger, classify as **fear**.
        - If it is about moral or physical revulsion, classify as **disgust**.
        - If the feeling is self-directed regarding a moral failing, classify as **guilt**.

        ### Shame
        - **Definition:** A feeling of humiliation or distress caused by the awareness of wrongdoing or foolish behavior, often accompanied by embarrassment, dishonor, or exposure to judgment.
        - **Examples:**
        - Feeling exposed or judged for personal flaws or mistakes.
        - Experiencing social disapproval or loss of reputation.
        - Being caught in acts of dishonesty or deceit.
        - Violating personal or cultural values, leading to feelings of unworthiness.
        - **Refinements:**
        - If the emotion focuses on feeling judged or humiliated without explicit remorse for harm, classify as **shame**.
        - If the emotion includes explicit remorse for harming someone, classify as **guilt**.

        ### Guilt
        - **Definition:** A feeling of remorse or regret over actions that have harmed others or violated moral principles, focused on personal responsibility rather than external judgment.
        - **Examples:**
        - Feeling bad for hurting someone’s feelings.
        - Regretting a past mistake or unethical action.
        - Realizing that one’s negligence caused harm.
        - Breaking a rule and later feeling remorseful.
        - **Refinements:**
        - Only classify as **guilt** if there is explicit harm or clear wrongdoing causing remorse.
        - If there is no explicit harm mentioned, default to **shame**.
        - Increase the threshold for guilt; use it only when clear personal responsibility for harm is evident.

        ---

        ## Additional Guidelines:
        - **Default to Shame:** When the text does not explicitly show remorse for an action but hints at personal inadequacy, classify as **shame**.
        - **Mixed Cases:** Evaluate the overall tone and select the most prominent emotion.
        - **Confidence Scoring:** Assign a confidence score between `0` and `1` based on your certainty. If confidence is below `0.6`, lean toward **shame** or **sadness**.
        - **Output Format:** Return your result strictly as a JSON object in the following structure::

        ```json
        {{
        "emotion": "one_of_the_specified_emotions",
        "confidence_score": confidence_value
        }}

        ## Final Task Instruction
        Classify the following text and output only the JSON response. No explanations, no additional formatting, and no <think> tags.

        Text: {row.text}

        Reminder: Output must be strictly JSON. Any deviation from this format is not allowed.
        """
    labels = ['joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', 'guilt']
    if row['predicted_emotion_deepseek'].lower() not in labels or row['emotion']!= row['predicted_emotion_deepseek'].lower():
        messages = [
            {
                'role': 'user',
                'content': prompt,
            }
        ]
        try:
            response: ChatResponse = chat(model='deepseek-r1', messages=messages, options={"temperature":0.3})
            
            content = response['message']['content']
            # Check if content is wrapped in markdown-like code block
            match = re.search(r'```json\s*(.*?)\s*```', content, re.DOTALL)
            if match:
                content = match.group(1).strip()
            # Parse the JSON response correctly
            predicted_emotion = json.loads(content)
            predicted_emotion_label  = "unknown"
            predicted_emotion_score = 0.0
            if predicted_emotion.get("emotion").lower() in labels:
                # Store the results in the DataFrame
                predicted_emotion_label = predicted_emotion.get('emotion', 'unknown').lower()
                predicted_emotion_score = predicted_emotion.get('confidence_score', 0.0)   
            df_isear.at[index, 'predicted_emotion_deepseek'] = predicted_emotion_label
            df_isear.at[index, 'confidence_score_deepseek'] = predicted_emotion_score 
        except:
            df_isear.at[index, 'predicted_emotion_deepseek'] = 'unknown'
            df_isear.at[index, 'confidence_score_deepseek'] = 0
        
        print(f"Row {index}| Ground truth - {row.emotion} and predicted -{predicted_emotion_label}")
        if index % 500==0:
            df_isear.to_csv('/Users/arasun/works/neuromatch/isear/isear_dataset.csv', index=False)
df_isear.to_csv('/Users/arasun/works/neuromatch/isear/isear_dataset.csv', index=False)

Row 5999| Ground truth - anger and predicted -anger
Row 6000| Ground truth - sadness and predicted -sadness
Row 6001| Ground truth - disgust and predicted -disgust
Row 6002| Ground truth - shame and predicted -shame
Row 6003| Ground truth - guilt and predicted -guilt
Row 6004| Ground truth - joy and predicted -joy
Row 6005| Ground truth - fear and predicted -shame
Row 6006| Ground truth - anger and predicted -sadness
Row 6007| Ground truth - sadness and predicted -sadness
Row 6008| Ground truth - disgust and predicted -sadness
Row 6009| Ground truth - shame and predicted -guilt
Row 6010| Ground truth - guilt and predicted -shame
Row 6011| Ground truth - joy and predicted -joy
Row 6012| Ground truth - fear and predicted -anger
Row 6013| Ground truth - anger and predicted -anger
Row 6014| Ground truth - sadness and predicted -sadness
Row 6015| Ground truth - disgust and predicted -disgust
Row 6016| Ground truth - guilt and predicted -guilt
Row 6017| Ground truth - joy and predicted -joy


In [5]:
df_isear.to_csv('/Users/arasun/works/neuromatch/isear/isear_dataset.csv', index=False)

## Anthropic Claude model

In [ ]:
import json
import pandas as pd
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
import json
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel

# Generic Model Initialization
def initialize_model(provider="anthropic", temperature=0.3, max_tokens=200):
    if provider == "anthropic":
        return ChatAnthropic(model_name='claude-3-5-sonnet-20241022', temperature=temperature, max_tokens=max_tokens)
    elif provider == "openai":
        return ChatOpenAI(model_name='gpt-4o', temperature=temperature, max_tokens=max_tokens)
    else:
        raise ValueError("Unsupported provider")

# Define structured response schema using Pydantic
class EmotionRecord(BaseModel):
    emotion: str = "unknown"
    confidence_score: float = 0.0

def classify_emotions(df, provider="anthropic"):
    # Define JSON output parser
    json_parser = JsonOutputParser(pydantic_object=EmotionRecord)
    genai_model = initialize_model(provider)
    # Define the prompt template
    prompt_template = ChatPromptTemplate.from_template(
        """
        You are an expert in emotion analysis. Given a text input, classify the emotion **strictly** as one of the following:  
            **'joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', 'guilt'.**  
            **No other emotions are allowed.**  

        Here is the text to analyze:
        <input_text>
        {text}
        </input_text>

        ## Instructions:
        1. **Read and understand** the input text carefully.
        2. **Strictly classify** the text as one of these **only**: 'joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', or 'guilt'.

        ## **Definitions for the Labels:**

        ### **Joy**  
        A feeling of **happiness, excitement, or pleasure**, often from positive experiences, achievements, or pleasant interactions.  
        Examples:
            Receiving good news, celebrating success, spending time with loved ones.
            Enjoying a meaningful moment with friends or family.
        Refinement: If the text expresses relief rather than happiness, classify it as joy. 

        ### **Fear**  
        A response to **perceived danger, risk, or uncertainty**, often accompanied by **anxiety, nervousness, or distress**.  
        - Fear arises when an individual **feels threatened, helpless, or in danger**.  
        - It can be related to **personal safety, uncertain future, or overwhelming situations**.

        **Examples of fear:**
        - Experiencing a sudden scare or physical danger (e.g., a traffic accident, being chased).
        - Worrying about an uncertain future or an important exam.
        - Feeling vulnerable or unsafe in an unfamiliar or threatening environment.
        - Hearing about a serious illness or personal risk (e.g., medical tests, a pandemic).
        - Imagining a loved one being in danger.
        - Being caught in a situation where you have **no control**.

        **Refinement:**  
        - Fear vs. Sadness → If the emotion is about loss, classify it as sadness.
        - Fear vs. Guilt → If the emotion is about remorse for past actions, classify it as guilt.    

        ### **Anger**  
        A feeling of **irritation, frustration, or rage** in response to a **perceived injustice, wrongdoing, or moral outrage**.  
        **Anger often arises from unfairness, mistreatment, or feeling powerless to stop an unjust action.**  
        **Examples of anger:**
        - Witnessing someone being treated unfairly.
        - Feeling ignored, rejected, or ridiculed without justification.
        - Seeing cruelty, unethical behavior, or injustice.
        - Facing obstacles due to someone else's irresponsibility.
        - Being accused of something unfairly.
        - Personal attacks or insults
        - Discrimination/prejudice
        - Broken promises/trust
        - Invasion of personal boundaries
        - Feeling manipulated/controlled

        **Refinement:** 
        * Anger vs. Sadness → If the text shows resentment or frustration, classify it as anger. If it expresses grief or emotional pain, classify it as sadness.

        ### **Sadness**  
        A feeling of **deep sorrow, grief, or loss**, often associated with **disappointment, longing, or emotional pain** caused by separation, failure, or death.  

        **Examples of sadness:**  
        - Losing a loved one, pet, or cherished relationship.  
        - Feeling lonely or disconnected.  
        - Experiencing personal failure or missed opportunities.  
        - Witnessing suffering or misfortune of others.  
        - Regretting something that cannot be changed.  
        - Feeling rejected/abandoned
        - Nostalgia for better times
        - Loss of dreams/hopes
        - Social isolation/exclusion
        - Empathy for others' suffering
        **Refinement:**  
        - **Sadness vs. Guilt** → If the emotion is **regret about one’s own actions**, classify as **guilt**. If it is **about loss or despair**, classify as **sadness**.  
        - **Sadness vs. Fear** → If the feeling is **about uncertainty or potential danger**, classify as **fear**. If it is about **mourning something lost**, classify as **sadness**.  
        - **Sadness vs. Anger** → If the response is **due to injustice or frustration**, classify as **anger**. If it is **due to sorrow or emotional pain**, classify as **sadness**.    

        ### **Disgust**  
        **strong aversion, repulsion, or deep-seated rejection** toward something perceived as **offensive, impure, unethical, or physically revolting**.  
        - **Disgust often arises from:**  
        - Witnessing **violence, cruelty, or injustice**.
        - Seeing or **hearing about unethical, immoral, or corrupt actions**.
        - Experiencing **physical repulsion (e.g., rotten food, bad smells, grotesque sights)**.

        **Examples of disgust:**
        - Seeing animals or people being mistreated.
        - Reading about unethical political actions or human rights violations.
        - Witnessing something grotesque or disturbing in a movie.
        - Feeling sickened by someone's behavior (e.g., lying, cheating, betrayal).
        - Encountering **unclean, unhygienic, or repulsive physical conditions**.

        **Refinement:**  
        - **Disgust vs. Fear** → If the reaction is about **personal danger**, classify as **fear**. If it is **moral or physical revulsion**, classify as **disgust**.  
        - **Disgust vs. Sadness** → If the reaction is about **pity and sorrow**, classify as **sadness**. If it is **repulsion and rejection**, classify as **disgust**.  
        - **Disgust vs. Guilt** → If the feeling is **self-directed** and about moral failing, classify as **guilt**. If it is **revulsion toward an external event**, classify as **disgust**.   

        ### **Shame**  
        A feeling of **humiliation or distress** caused by the **consciousness of wrong or foolish behavior**, often associated with **embarrassment, dishonor, or feeling exposed** in the eyes of others.

        **Examples of shame:**  
        - Feeling **exposed or judged** by others for personal flaws or mistakes.  
        - Actions that bring about **social disapproval** or **loss of reputation**.  
        - Situations involving **lying, dishonesty, or deceit** when discovered by others.  
        - **Behavior that goes against personal or cultural values**, resulting in feeling unworthy or disgraced.  
        - Making mistakes in front of others/public failure
        - Being caught in dishonest/unethical behavior
        - Perceived violation of social/cultural norms
        - Feeling inadequate compared to others

        Refinement:

        - Shame vs. Guilt → If the person feels judged, exposed, or humiliated, classify as shame. If they feel remorse for harming someone, classify as guilt.
        - Shame vs. Sadness → If the focus is on embarrassment or judgment, classify as shame. If it is about loss or despair, classify as sadness.  

        ### **Guilt**  
        A feeling of **remorse or regret** over actions that have **harmed others or violated moral principles**. Unlike shame, **guilt is about personal responsibility rather than external judgment**.  

        **Examples of guilt:**  
        - Feeling bad for hurting someone’s feelings.  
        - Regretting a past mistake or unethical action.  
        - Realizing that one's negligence caused harm.  
        - Breaking a rule and later feeling remorseful.  
        - Feeling responsible for another’s misfortune.  
        - Not living up to responsibilities
        - Lying or deceiving others
        - Breaking promises
        - Failing to help someone in need
        - Going against personal values
        **Refinement:**  
        - **Guilt vs. Shame** → If the emotion is **internally focused on self-worth and social judgment**, classify as **shame**. If it is about **personal responsibility for a specific action**, classify as **guilt**.  
        - **Guilt vs. Sadness** → If the emotion is **about loss**, classify as **sadness**. If it is **about regret for one’s own actions**, classify as **guilt**.  
        - **Guilt vs. Fear** → If the concern is about **future consequences**, classify as **fear**. If it is **about remorse for past actions**, classify as **guilt**.  
        ---

        ## **Classification Guidelines**
        - **Choose the most prominent emotion** when multiple emotions are present. Prioritize the **strongest** emotional response conveyed in the text.  
        - **Restrict classifications to the predefined categories**: **anger, disgust, fear, guilt, joy, sadness, and shame.** Avoid introducing new emotions (e.g., `"neutral," "frustration," "jealousy," "relief," "curiosity," "surprise"`).  
        - **Map related emotions to the closest predefined category**:
        - **Frustration → Anger**  
        - **Disappointment → Sadness**  
        - **Anxiety → Fear**  
        - **Embarrassment → Shame**  
        - **Assign a confidence score** (`0` to `1`) reflecting the certainty of the classification. Higher scores indicate greater confidence.  
        - **Evaluate the emotional context thoroughly**: Assess **tone, intensity, and underlying sentiment** to ensure accurate classification. Prioritize the emotion that most strongly aligns with the context.  
        - **Maintain consistency across similar inputs**: If an entry is ambiguous, compare it with similar previously classified entries to ensure uniformity in classification. 
        
        ## **Shame vs. Guilt: Decision Rules**
        - **Shame**: If the emotion is about **humiliation, dishonor, embarrassment, or feeling exposed to judgment**, classify as **shame**.
        - `"When I did not speak the truth."` → **Shame** (because no explicit remorse is expressed)
        - `"I was caught lying."` → **Shame** (Social exposure)
        - `"I was caught stealing."` → **Shame** (Social humiliation)
        - `"When I forgot my mother’s birthday."` → **Shame** (Embarrassment)
        
        - **Guilt**: If the emotion is **about personal responsibility** for harming others, classify as **guilt**.
        - `"I lied to my friend and felt bad."` → **Guilt**
        - `"I hurt my girlfriend by yelling at her."` → **Guilt**
        - `"I cheated on my partner."` → **Guilt**
        
        ---
        ## **3️⃣ Handling Ambiguous Cases**
        If the text **does not explicitly** show remorse over an action but hints at personal inadequacy, **default to shame**.
        - Example: `"The discovery that one has only thought of oneself."`  
        - No direct action harming others → **Shame**
        - Example: `"I drank wine too often."`  
        - No explicit harm caused → **Shame**
        - Example: `"I felt bad after I yelled at my girlfriend."`  
        - Explicit regret for action → **Guilt**

        ---
        ## **4️⃣ Confidence Scoring Adjustments**
        - **Increase the threshold for guilt.**  
        - Guilt classification **must** include a **clear action that caused harm.**
        - **Low-confidence cases default to shame.**  
        - If the sentence lacks **explicit harm** → classify as **shame**.

        ---
        ### **Example Corrections with the New Prompt**
        | Text  | **Old Classification** | **New Classification** | **Reason for Fix** |
        |--------|------------------|------------------|----------------|
        | `"When I did not speak the truth."` | Guilt | Shame | No explicit remorse, just self-reflection |
        | `"I was caught lying."` | Guilt | Shame | Public exposure, humiliation |
        | `"I promised to help a friend and did not keep the promise."` | Guilt | | Explicit remorse for broken promise |
        | `"I forgot my mother's birthday."` | Guilt | Shame | Social embarrassment, not harm |
        | `"I cheated on a test in 7th grade."` | Guilt | | Clear wrongdoing |
        | `"When I drank wine too often."` | Guilt | Shame | Personal inadequacy, no harm to others |

        ---

        ### **Expected Improvements**
        **Reduced false negatives:** Shame cases will no longer be misclassified as guilt.  
        **Lower hallucination risk:** Ambiguous cases default to shame instead of hallucinating guilt.  
        **More consistent classifications:** Social exposure vs. moral wrongdoing is clearly separated.

        ### Output Format:
        - Return only a valid JSON object with no additional formatting, code blocks, or explanations.
        - Ensure the output follows this structure:

        {{
          "emotion": "one_of_the_specified_emotions",
          "confidence_score": confidence_value
        }}

        Now, analyze the text and output only the JSON response with no additional formatting, code blocks, or explanations.
        """
    )

    # Define column names dynamically
    confidence_col = f"confidence_score_{provider}"
    emotion_col = f"predicted_emotion_{provider}"
    # Combine components into a pipeline (LangChain runnable sequence)
    llm_chain = prompt_template | genai_model | json_parser
    # Ensure 'confidence_score_openai' column exists and is of type float
    if confidence_col not in df.columns:
        df[confidence_col] = 0.0  # Default to float
    df[confidence_col] = df[confidence_col].astype(float) 
    labels = ['joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', 'guilt']
    for index, row in df.iterrows():
        if (row[emotion_col] not in labels) or (row[emotion_col]!=row['emotion']):
            try:
                # Prepare input text
                input_text = {"text": row["text"]}

                # Invoke model using LangChain pipeline
                predicted_emotion = llm_chain.invoke(input_text)
                if predicted_emotion.get("emotion") not in labels:
                    df.at[index, emotion_col] = "unknown"
                    df.at[index, confidence_col] = 0.0
                else:
                    # Store results in DataFrame
                    df.at[index, emotion_col] = predicted_emotion.get("emotion")
                    df.at[index, confidence_col] = float(predicted_emotion.get("confidence_score", 0.0))

            except Exception as e:
                print(f"Error processing row {index}: {e}")
                df.at[index, emotion_col] = "unknown"
                df.at[index, confidence_col] = 0.0

            print(f"Row {index}| Ground truth - {row['emotion']} | Predicted - {df.at[index,emotion_col]}")

    return df

## OpenAI model model

In [ ]:
df_isear = pd.read_csv('isear_dataset.csv')
df_isear = classify_emotions(df_isear, provider="openai")
df_isear.to_csv('isear_dataset.csv', index=False)

## Comparing model performance

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# Load dataset
df = pd.read_csv("/Users/arasun/works/neuromatch/isear/isear_dataset.csv")  # Change to the correct file path

# Define models to compare
models = ["llama 3.2", "OpenAI", "Claude","llama 3.3", "Qwen","Cohere R+","DeepSeek"]
model_columns = ["predicted_emotion_llama32", "predicted_emotion_openai", "predicted_emotion_anthropic","predicted_emotion_llama3_3", "predicted_emotion_qwen", "predicted_emotion_cohere","predicted_emotion_deepseek"]

# Compute evaluation metrics
def evaluate_model(y_true, y_pred, model_name):
    report = classification_report(y_true, y_pred, output_dict=True)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    cm = confusion_matrix(y_true, y_pred)
    
    print(f"\nModel: {model_name}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}\n")
    print(classification_report(y_true, y_pred))
    
    return report, accuracy, precision, recall, f1, cm

metrics = {}
conf_matrices = {}

for model_name, model_col in zip(models, model_columns):
    metrics[model_name], acc, prec, rec, f1, conf_matrices[model_name] = evaluate_model(df["emotion"].str.lower(), df[model_col].str.lower(), model_name)

# Convert metrics into DataFrame for comparison
metric_df = pd.DataFrame({
    model: {
        "Accuracy": metrics[model]["accuracy"],
        "Precision": metrics[model]["weighted avg"]["precision"],
        "Recall": metrics[model]["weighted avg"]["recall"],
        "F1-score": metrics[model]["weighted avg"]["f1-score"]
    } for model in models
})
# Plot Accuracy, Precision, Recall, and F1-score
metric_df.T.plot(kind='bar', figsize=(10, 6), colormap="viridis", rot=0)
plt.title("Model Performance Comparison")
plt.ylabel("Score")
plt.ylim(0, 1)
plt.legend(loc="lower right")
plt.show()

# Additional comparative plots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
metrics_list = ["Accuracy", "Precision", "Recall", "F1-score"]
colors = sns.color_palette("viridis", len(models))

for ax, metric in zip(axes.flat, metrics_list):
    sns.barplot(x=models, y=metric_df.loc[metric], ax=ax, palette=colors)
    ax.set_title(f"Comparison of {metric}")
    ax.set_ylabel(metric)
    ax.set_ylim(0, 1)

plt.tight_layout()
plt.show()

# Line plot for trend comparison
plt.figure(figsize=(10, 6))
for model in models:
    plt.plot(metric_df.index, metric_df[model], marker='o', label=model)
plt.title("Trend Comparison of Model Performance")
plt.ylabel("Score")
plt.xlabel("Metrics")
plt.ylim(0, 1)
plt.legend()
plt.grid()
plt.show()

# Plot Confusion Matrices
for model, cm in conf_matrices.items():
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(df["emotion"]), yticklabels=np.unique(df["emotion"]))
    plt.title(f"Confusion Matrix - {model}")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.show()


In [ ]:
df_isear.to_csv('/Users/arasun/works/neuromatch/isear/isear_dataset.csv', index=False)